# 📘 Hindi QA using FastText

In [ ]:
# ✅ Install Required Libraries
!pip install -q nltk rouge-score scikit-learn fasttext

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# ✅ Imports
import json, csv, re, gzip
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/M Tech/Sem3")

Mounted at /content/drive


In [ ]:
# ✅ Load and Flatten SQuAD-style Hindi QA Dataset
def load_and_flatten(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    samples = []
    for domain in data['domains']:
        for ctx in domain['contexts']:
            context = ctx['context']
            for qa in ctx['qas']:
                question = qa['question']
                answer_text = qa['answer']
                answer_start = context.find(answer_text)
                if answer_start == -1:
                    continue
                samples.append({
                    'id': qa['id'],
                    'context': context,
                    'question': question,
                    'answer_text': answer_text,
                    'answer_start': answer_start
                })
    return samples

In [ ]:
# ✅ Load FastText Embeddings
def load_fasttext_vectors(path, vocab_limit=500000):
    embeddings = {}
    open_func = gzip.open if path.endswith('.gz') else open
    with open_func(path, 'rt', encoding='utf-8') as f:
        next(f)
        for i, line in enumerate(f):
            if i >= vocab_limit:
                break
            parts = line.strip().split()
            word = parts[0]
            vec = np.array(parts[1:], dtype='float32')
            embeddings[word] = vec
    print(f"Loaded {len(embeddings)} word vectors.")
    return embeddings

In [ ]:
# ✅ Hindi Sentence Tokenizer
def hindi_sent_tokenize(text):
    sentence_endings = re.compile(r'(?<=[।!?])\s+')
    sentences = sentence_endings.split(text.strip())
    return [s.strip() for s in sentences if s.strip()]

In [ ]:
# ✅ Sentence to Vector
def sentence_to_vec(sentence, embeddings, dim=300):
    words = sentence.strip().split()
    vecs = [embeddings[word] for word in words if word in embeddings]
    return np.mean(vecs, axis=0) if vecs else np.zeros(dim)

In [ ]:
# ✅ Get Best Matching Sentence as Answer
def get_best_answer(context, question, embeddings, dim=300):
    sentences = hindi_sent_tokenize(context)
    q_vec = sentence_to_vec(question, embeddings, dim)
    s_vecs = [sentence_to_vec(s, embeddings, dim) for s in sentences]
    if not any(np.any(v) for v in s_vecs):
        return ""
    sims = cosine_similarity([q_vec], s_vecs)[0]
    return sentences[int(np.argmax(sims))]

In [ ]:
# ✅ Evaluation Metrics
def calculate_metrics(true_answer, predicted_answer):
    smooth = SmoothingFunction().method1
    bleu = sentence_bleu([true_answer.split()], predicted_answer.split(), smoothing_function=smooth)
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_l = scorer.score(true_answer, predicted_answer)['rougeL'].fmeasure
    true_words = set(true_answer.split())
    pred_words = set(predicted_answer.split())
    common = true_words.intersection(pred_words)
    precision = len(common) / len(pred_words) if pred_words else 0
    recall = len(common) / len(true_words) if true_words else 0
    f1 = 2 * precision * recall / (precision + recall + 1e-8) if (precision + recall) > 0 else 0
    return {"BLEU": bleu, "ROUGE-L": rouge_l, "F1": f1}

In [ ]:
# ✅ Predict and Save Results + Metrics
def predict_and_evaluate(samples, embeddings, save_prefix="predictions_fasttext", dim=300):
    predictions = []
    all_bleu, all_rouge, all_f1 = [], [], []
    for i, sample in enumerate(samples):
        question, context, true_answer = sample['question'], sample['context'], sample['answer_text']
        predicted_answer = get_best_answer(context, question, embeddings, dim)
        scores = calculate_metrics(true_answer, predicted_answer)
        predictions.append({
            "id": sample['id'],
            "question": question,
            "context": context,
            "true_answer": true_answer,
            "predicted_answer": predicted_answer,
            "bleu": scores["BLEU"],
            "rouge_l": scores["ROUGE-L"],
            "f1": scores["F1"]
        })
        all_bleu.append(scores["BLEU"])
        all_rouge.append(scores["ROUGE-L"])
        all_f1.append(scores["F1"])
        if i % 100 == 0:
            print(f"[{i}/{len(samples)}] Q: {question}\n→ Predicted: {predicted_answer}\n")
    # Save to JSON
    with open(f"{save_prefix}.json", "w", encoding="utf-8") as jf:
        json.dump(predictions, jf, indent=2, ensure_ascii=False)
    # Save to CSV
    with open(f"{save_prefix}.csv", "w", encoding="utf-8", newline='') as cf:
        writer = csv.DictWriter(cf, fieldnames=predictions[0].keys())
        writer.writeheader()
        writer.writerows(predictions)
    print("\n📊 Average Scores:")
    print(f"BLEU: {np.mean(all_bleu):.4f}, ROUGE-L: {np.mean(all_rouge):.4f}, F1: {np.mean(all_f1):.4f}")
    return predictions

In [ ]:
# Load dataset and embeddings
val_data = load_and_flatten("Dataset/validation.json")
embeddings = load_fasttext_vectors("Embeddings/cc.hi.300.vec")

# Run prediction + metrics
predict_and_evaluate(val_data, embeddings, save_prefix="Embeddings/val_predictions_fasttext")


Loaded 500000 word vectors.
[0/395] Q: क्या क्रीं कुंड तक पहुँचने के लिए सार्वजनिक परिवहन उपलब्ध है?
→ Predicted: वाराणसी शहर के प्रमुख स्थानों से क्रीं कुंड तक पहुँचने के लिए विभिन्न सार्वजनिक परिवहन विकल्प उपलब्ध हैं।

[100/395] Q: अगस्त्येश्वर मंदिर बनारस रेलवे स्टेशन से कितनी दूरी पर है?
→ Predicted: अगस्त्येश्वर मंदिर बनारस रेलवे स्टेशन से 5 किलोमीटर की दूरी पर स्थित है।

[200/395] Q: क्या दुर्ग विनायक मंदिर में गणेश चतुर्थी के दिन विशेष आरती का आयोजन होता है?
→ Predicted: हाँ, दुर्ग विनायक मंदिर में गणेश चतुर्थी के दिन विशेष आरती का आयोजन होता है।

[300/395] Q: क्या श्री काशी अन्नपूर्णा वासवी आर्य व्यास वृद्धाश्रम एवं अन्नदान क्षेत्र में भोजन की सुविधा है?
→ Predicted: . हाँ, श्री काशी अन्नपूर्णा वासवी आर्य व्यास वृद्धाश्रम एवं अन्नदान क्षेत्र में भोजन की सुविधा उपलब्ध है।


📊 Average Scores:
BLEU: 0.8402, ROUGE-L: 0.4017, F1: 0.8874


[{'id': 'kund_980',
  'question': 'क्या क्रीं कुंड तक पहुँचने के लिए सार्वजनिक परिवहन उपलब्ध है?',
  'context': 'हाँ, क्रीं कुंड तक पहुँचने के लिए बस, ऑटो-रिक्शा और टैक्सी की सुविधा उपलब्ध है। वाराणसी शहर के प्रमुख स्थानों से क्रीं कुंड तक पहुँचने के लिए विभिन्न सार्वजनिक परिवहन विकल्प उपलब्ध हैं। श्रद्धालु और पर्यटक इन वाहनों का उपयोग करके आराम से क्रीं कुंड तक पहुँच सकते हैं। ऑटो-रिक्शा और टैक्सी विशेष रूप से क्रीं कुंड जैसे धार्मिक स्थलों तक सीधे पहुँचने के लिए सुविधाजनक होते हैं। इसके अलावा, शहर के अन्य हिस्सों से बस सेवाएं भी चलती हैं, जो क्रीं कुंड तक पहुँचने का एक अच्छा विकल्प प्रदान करती हैं।',
  'true_answer': 'हाँ, क्रीं कुंड तक पहुँचने के लिए बस, ऑटो-रिक्शा और टैक्सी की सुविधा उपलब्ध है।',
  'predicted_answer': 'वाराणसी शहर के प्रमुख स्थानों से क्रीं कुंड तक पहुँचने के लिए विभिन्न सार्वजनिक परिवहन विकल्प उपलब्ध हैं।',
  'bleu': 0.2749977595322415,
  'rouge_l': 0,
  'f1': 0.43749999501953124},
 {'id': 'kund_1415',
  'question': 'क्या नाग कुआँ में स्नान करने की अनुमति है?',
  

In [ ]:
def evaluate_only(samples, embeddings, dim=300):
    from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
    from rouge_score import rouge_scorer

    def calculate_metrics(true_answer, predicted_answer):
        smooth = SmoothingFunction().method1
        bleu = sentence_bleu([true_answer.split()], predicted_answer.split(), smoothing_function=smooth)
        scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        rouge_l = scorer.score(true_answer, predicted_answer)['rougeL'].fmeasure
        true_words = set(true_answer.split())
        pred_words = set(predicted_answer.split())
        common = true_words.intersection(pred_words)
        if not true_words or not pred_words:
            return {"BLEU": 0.0, "ROUGE-L": 0.0, "F1": 0.0}
        precision = len(common) / len(pred_words)
        recall = len(common) / len(true_words)
        f1 = 2 * precision * recall / (precision + recall + 1e-8)
        return {"BLEU": bleu, "ROUGE-L": rouge_l, "F1": f1}

    bleu_scores, rouge_scores, f1_scores = [], [], []

    for i, sample in enumerate(samples):
        context = sample["context"]
        question = sample["question"]
        true_answer = sample["answer_text"]
        pred_answer = get_best_answer(context, question, embeddings, dim)
        scores = calculate_metrics(true_answer, pred_answer)
        bleu_scores.append(scores["BLEU"])
        rouge_scores.append(scores["ROUGE-L"])
        f1_scores.append(scores["F1"])

    # Final averages
    print("📊 Average Scores:")
    print(f"BLEU:     {np.mean(bleu_scores):.4f}")
    print(f"ROUGE-L:  {np.mean(rouge_scores):.4f}")
    print(f"F1 Score: {np.mean(f1_scores):.4f}")


In [ ]:
# Load your validation data and FastText embeddings
val_data = load_and_flatten("Dataset/validation.json")
embeddings = load_fasttext_vectors("Embeddings/cc.hi.300.vec")

# Just print evaluation scores
evaluate_only(val_data, embeddings)

Loaded 500000 word vectors.
📊 Average Scores:
BLEU:     0.8402
ROUGE-L:  0.4017
F1 Score: 0.8874
